<a href="https://colab.research.google.com/github/juangui96/DS-_-Boot-Camp/blob/main/Avance_Proyecto_VI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##SUPER STORE - PREDICCION Y ANALISIS
Objetivo: Complementar el analisis por region (estado, ciudad) y fecha de las principales busquedas en internet mediante la API de Google Trends, utilizando como keywords las subcategorias de los productos.


##Cargue Librerias

In [1]:
#importar librerias
import pandas as pd
import numpy as np


In [31]:
! pip install chart_studio
import plotly.express as px
import plotly.graph_objects as go
import chart_studio.plotly as py
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
#ignorar warning

import warnings
warnings.filterwarnings("ignore")

In [4]:
!pip install pytrends

In [5]:
#pyptrends y funciones requerdias
import json
import pandas as pd
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from requests import status_codes

from pytrends import exceptions

from urllib.parse import quote

BASE_TRENDS_URL = 'https://trends.google.com/trends'


class TrendReq(object):
    """
    Google Trends API
    """
    GET_METHOD = 'get'
    POST_METHOD = 'post'
    GENERAL_URL = f'{BASE_TRENDS_URL}/api/explore'
    INTEREST_OVER_TIME_URL = f'{BASE_TRENDS_URL}/api/widgetdata/multiline'
    MULTIRANGE_INTEREST_OVER_TIME_URL = f'{BASE_TRENDS_URL}/api/widgetdata/multirange'
    INTEREST_BY_REGION_URL = f'{BASE_TRENDS_URL}/api/widgetdata/comparedgeo'
    RELATED_QUERIES_URL = f'{BASE_TRENDS_URL}/api/widgetdata/relatedsearches'
    TRENDING_SEARCHES_URL = f'{BASE_TRENDS_URL}/hottrends/visualize/internal/data'
    TOP_CHARTS_URL = f'{BASE_TRENDS_URL}/api/topcharts'
    SUGGESTIONS_URL = f'{BASE_TRENDS_URL}/api/autocomplete/'
    CATEGORIES_URL = f'{BASE_TRENDS_URL}/api/explore/pickers/category'
    TODAY_SEARCHES_URL = f'{BASE_TRENDS_URL}/api/dailytrends'
    REALTIME_TRENDING_SEARCHES_URL = f'{BASE_TRENDS_URL}/api/realtimetrends'
    ERROR_CODES = (500, 502, 504, 429)

    def __init__(self, hl='en-US', tz=360, geo='', timeout=(2, 5), proxies='',
                 retries=0, backoff_factor=0, requests_args=None):
        """
        Initialize default values for params
        """
        # google rate limit
        self.google_rl = 'You have reached your quota limit. Please try again later.'
        self.results = None
        # set user defined options used globally
        self.tz = tz
        self.hl = hl
        self.geo = geo
        self.kw_list = list()
        self.timeout = timeout
        self.proxies = proxies  # add a proxy option
        self.retries = retries
        self.backoff_factor = backoff_factor
        self.proxy_index = 0
        self.requests_args = requests_args or {}
        self.cookies = self.GetGoogleCookie()
        # intialize widget payloads
        self.token_payload = dict()
        self.interest_over_time_widget = dict()
        self.interest_by_region_widget = dict()
        self.related_topics_widget_list = list()
        self.related_queries_widget_list = list()

        self.headers = {'accept-language': self.hl}
        self.headers.update(self.requests_args.pop('headers', {}))

    def GetGoogleCookie(self):
        """
        Gets google cookie (used for each and every proxy; once on init otherwise)
        Removes proxy from the list on proxy error
        """
        while True:
            if "proxies" in self.requests_args:
                try:
                    return dict(filter(lambda i: i[0] == 'NID', requests.post(
                        f'{BASE_TRENDS_URL}/?geo={self.hl[-2:]}',
                        timeout=self.timeout,
                        **self.requests_args
                    ).cookies.items()))
                except:
                    continue
            else:
                if len(self.proxies) > 0:
                    proxy = {'https': self.proxies[self.proxy_index]}
                else:
                    proxy = ''
                try:
                    return dict(filter(lambda i: i[0] == 'NID', requests.post(
                        f'{BASE_TRENDS_URL}/?geo={self.hl[-2:]}',
                        timeout=self.timeout,
                        proxies=proxy,
                        **self.requests_args
                    ).cookies.items()))
                except requests.exceptions.ProxyError:
                    print('Proxy error. Changing IP')
                    if len(self.proxies) > 1:
                        self.proxies.remove(self.proxies[self.proxy_index])
                    else:
                        print('No more proxies available. Bye!')
                        raise
                    continue

    def GetNewProxy(self):
        """
        Increment proxy INDEX; zero on overflow
        """
        if self.proxy_index < (len(self.proxies) - 1):
            self.proxy_index += 1
        else:
            self.proxy_index = 0

    def _get_data(self, url, method=GET_METHOD, trim_chars=0, **kwargs):
        """Send a request to Google and return the JSON response as a Python object
        :param url: the url to which the request will be sent
        :param method: the HTTP method ('get' or 'post')
        :param trim_chars: how many characters should be trimmed off the beginning of the content of the response
            before this is passed to the JSON parser
        :param kwargs: any extra key arguments passed to the request builder (usually query parameters or data)
        :return:
        """
        s = requests.session()
        # Retries mechanism. Activated when one of statements >0 (best used for proxy)
        if self.retries > 0 or self.backoff_factor > 0:
            retry = Retry(total=self.retries, read=self.retries,
                          connect=self.retries,
                          backoff_factor=self.backoff_factor,
                          status_forcelist=TrendReq.ERROR_CODES,
                          allowed_methods=frozenset(['GET', 'POST']))
            s.mount('https://', HTTPAdapter(max_retries=retry))

        s.headers.update(self.headers)
        if len(self.proxies) > 0:
            self.cookies = self.GetGoogleCookie()
            s.proxies.update({'https': self.proxies[self.proxy_index]})
        if method == TrendReq.POST_METHOD:
            response = s.post(url, timeout=self.timeout,
                              cookies=self.cookies, **kwargs,
                              **self.requests_args)  # DO NOT USE retries or backoff_factor here
        else:
            response = s.get(url, timeout=self.timeout, cookies=self.cookies,
                             **kwargs, **self.requests_args)  # DO NOT USE retries or backoff_factor here
        # check if the response contains json and throw an exception otherwise
        # Google mostly sends 'application/json' in the Content-Type header,
        # but occasionally it sends 'application/javascript
        # and sometimes even 'text/javascript
        if response.status_code == 200 and 'application/json' in \
                response.headers['Content-Type'] or \
                'application/javascript' in response.headers['Content-Type'] or \
                'text/javascript' in response.headers['Content-Type']:
            # trim initial characters
            # some responses start with garbage characters, like ")]}',"
            # these have to be cleaned before being passed to the json parser
            content = response.text[trim_chars:]
            # parse json
            self.GetNewProxy()
            return json.loads(content)
        else:
            if response.status_code == status_codes.codes.too_many_requests:
                raise exceptions.TooManyRequestsError.from_response(response)
            raise exceptions.ResponseError.from_response(response)

    def build_payload(self, kw_list, cat=0, timeframe='today 5-y', geo='',
                      gprop=''):
        """Create the payload for related queries, interest over time and interest by region"""
        if gprop not in ['', 'images', 'news', 'youtube', 'froogle']:
            raise ValueError('gprop must be empty (to indicate web), images, news, youtube, or froogle')
        self.kw_list = kw_list
        self.geo = geo or self.geo
        self.token_payload = {
            'hl': self.hl,
            'tz': self.tz,
            'req': {'comparisonItem': [], 'category': cat, 'property': gprop}
        }

        # Check if timeframe is a list
        if isinstance(timeframe, list):
            for index, kw in enumerate(self.kw_list):
                keyword_payload = {'keyword': kw, 'time': timeframe[index], 'geo': self.geo}
                self.token_payload['req']['comparisonItem'].append(keyword_payload)
        else:
            # build out json for each keyword with
            for kw in self.kw_list:
                keyword_payload = {'keyword': kw, 'time': timeframe, 'geo': self.geo}
                self.token_payload['req']['comparisonItem'].append(keyword_payload)

        # requests will mangle this if it is not a string
        self.token_payload['req'] = json.dumps(self.token_payload['req'])
        # get tokens
        self._tokens()
        return

    def _tokens(self):
        """Makes request to Google to get API tokens for interest over time, interest by region and related queries"""
        # make the request and parse the returned json
        widget_dicts = self._get_data(
            url=TrendReq.GENERAL_URL,
            method=TrendReq.GET_METHOD,
            params=self.token_payload,
            trim_chars=4,
        )['widgets']
        # order of the json matters...
        first_region_token = True
        # clear self.related_queries_widget_list and self.related_topics_widget_list
        # of old keywords'widgets
        self.related_queries_widget_list[:] = []
        self.related_topics_widget_list[:] = []
        # assign requests
        for widget in widget_dicts:
            if widget['id'] == 'TIMESERIES':
                self.interest_over_time_widget = widget
            if widget['id'] == 'GEO_MAP' and first_region_token:
                self.interest_by_region_widget = widget
                first_region_token = False
            # response for each term, put into a list
            if 'RELATED_TOPICS' in widget['id']:
                self.related_topics_widget_list.append(widget)
            if 'RELATED_QUERIES' in widget['id']:
                self.related_queries_widget_list.append(widget)
        return

    def interest_over_time(self):
        """Request data from Google's Interest Over Time section and return a dataframe"""

        over_time_payload = {
            # convert to string as requests will mangle
            'req': json.dumps(self.interest_over_time_widget['request']),
            'token': self.interest_over_time_widget['token'],
            'tz': self.tz
        }

        # make the request and parse the returned json
        req_json = self._get_data(
            url=TrendReq.INTEREST_OVER_TIME_URL,
            method=TrendReq.GET_METHOD,
            trim_chars=5,
            params=over_time_payload,
        )

        df = pd.DataFrame(req_json['default']['timelineData'])
        if (df.empty):
            return df

        df['date'] = pd.to_datetime(df['time'].astype(dtype='float64'),
                                    unit='s')
        df = df.set_index(['date']).sort_index()
        # split list columns into seperate ones, remove brackets and split on comma
        result_df = df['value'].apply(lambda x: pd.Series(
            str(x).replace('[', '').replace(']', '').split(',')))
        # rename each column with its search term, relying on order that google provides...
        for idx, kw in enumerate(self.kw_list):
            # there is currently a bug with assigning columns that may be
            # parsed as a date in pandas: use explicit insert column method
            result_df.insert(len(result_df.columns), kw,
                             result_df[idx].astype('int'))
            del result_df[idx]

        if 'isPartial' in df:
            # make other dataframe from isPartial key data
            # split list columns into seperate ones, remove brackets and split on comma
            df = df.fillna(False)
            result_df2 = df['isPartial'].apply(lambda x: pd.Series(
                str(x).replace('[', '').replace(']', '').split(',')))
            result_df2.columns = ['isPartial']
            # Change to a bool type.
            result_df2.isPartial = result_df2.isPartial == 'True'
            # concatenate the two dataframes
            final = pd.concat([result_df, result_df2], axis=1)
        else:
            final = result_df
            final['isPartial'] = False

        return final

    def multirange_interest_over_time(self):
        """Request data from Google's Interest Over Time section across different time ranges and return a dataframe"""

        over_time_payload = {
            # convert to string as requests will mangle
            'req': json.dumps(self.interest_over_time_widget['request']),
            'token': self.interest_over_time_widget['token'],
            'tz': self.tz
        }

        # make the request and parse the returned json
        req_json = self._get_data(
            url=TrendReq.MULTIRANGE_INTEREST_OVER_TIME_URL,
            method=TrendReq.GET_METHOD,
            trim_chars=5,
            params=over_time_payload,
        )

        df = pd.DataFrame(req_json['default']['timelineData'])
        if (df.empty):
            return df

        result_df = pd.json_normalize(df['columnData'])

        # Split dictionary columns into seperate ones
        for i, column in enumerate(result_df.columns):
            result_df["[" + str(i) + "] " + str(self.kw_list[i]) + " date"] = result_df[i].apply(pd.Series)["formattedTime"]
            result_df["[" + str(i) + "] " + str(self.kw_list[i]) + " value"] = result_df[i].apply(pd.Series)["value"]
            result_df = result_df.drop([i], axis=1)

        # Adds a row with the averages at the top of the dataframe
        avg_row = {}
        for i, avg in enumerate(req_json['default']['averages']):
            avg_row["[" + str(i) + "] " + str(self.kw_list[i]) + " date"] = "Average"
            avg_row["[" + str(i) + "] " + str(self.kw_list[i]) + " value"] = req_json['default']['averages'][i]

        result_df.loc[-1] = avg_row
        result_df.index = result_df.index + 1
        result_df = result_df.sort_index()

        return result_df


    def interest_by_region(self, resolution='COUNTRY', inc_low_vol=False,
                           inc_geo_code=False):
        """Request data from Google's Interest by Region section and return a dataframe"""

        # make the request
        region_payload = dict()
        if self.geo == '':
            self.interest_by_region_widget['request'][
                'resolution'] = resolution
        elif self.geo == 'US' and resolution in ['DMA', 'CITY', 'REGION']:
            self.interest_by_region_widget['request'][
                'resolution'] = resolution

        self.interest_by_region_widget['request'][
            'includeLowSearchVolumeGeos'] = inc_low_vol

        # convert to string as requests will mangle
        region_payload['req'] = json.dumps(
            self.interest_by_region_widget['request'])
        region_payload['token'] = self.interest_by_region_widget['token']
        region_payload['tz'] = self.tz

        # parse returned json
        req_json = self._get_data(
            url=TrendReq.INTEREST_BY_REGION_URL,
            method=TrendReq.GET_METHOD,
            trim_chars=5,
            params=region_payload,
        )
        df = pd.DataFrame(req_json['default']['geoMapData'])
        if (df.empty):
            return df

        # rename the column with the search keyword
        geo_column = 'geoCode' if 'geoCode' in df.columns else 'coordinates'
        columns = ['geoName', geo_column, 'value']
        df = df[columns].set_index(['geoName']).sort_index()
        # split list columns into separate ones, remove brackets and split on comma
        result_df = df['value'].apply(lambda x: pd.Series(
            str(x).replace('[', '').replace(']', '').split(',')))
        if inc_geo_code:
            if geo_column in df.columns:
                result_df[geo_column] = df[geo_column]
            else:
                print('Could not find geo_code column; Skipping')

        # rename each column with its search term
        for idx, kw in enumerate(self.kw_list):
            result_df[kw] = result_df[idx].astype('int')
            del result_df[idx]

        return result_df

    def related_topics(self):
        """Request data from Google's Related Topics section and return a dictionary of dataframes
        If no top and/or rising related topics are found, the value for the key "top" and/or "rising" will be None
        """

        # make the request
        related_payload = dict()
        result_dict = dict()
        for request_json in self.related_topics_widget_list:
            # ensure we know which keyword we are looking at rather than relying on order
            try:
                kw = request_json['request']['restriction'][
                    'complexKeywordsRestriction']['keyword'][0]['value']
            except KeyError:
                kw = ''
            # convert to string as requests will mangle
            related_payload['req'] = json.dumps(request_json['request'])
            related_payload['token'] = request_json['token']
            related_payload['tz'] = self.tz

            # parse the returned json
            req_json = self._get_data(
                url=TrendReq.RELATED_QUERIES_URL,
                method=TrendReq.GET_METHOD,
                trim_chars=5,
                params=related_payload,
            )

            # top topics
            try:
                top_list = req_json['default']['rankedList'][0]['rankedKeyword']
                df_top = pd.json_normalize(top_list, sep='_')
            except KeyError:
                # in case no top topics are found, the lines above will throw a KeyError
                df_top = None

            # rising topics
            try:
                rising_list = req_json['default']['rankedList'][1]['rankedKeyword']
                df_rising = pd.json_normalize(rising_list, sep='_')
            except KeyError:
                # in case no rising topics are found, the lines above will throw a KeyError
                df_rising = None

            result_dict[kw] = {'rising': df_rising, 'top': df_top}
        return result_dict

    def related_queries(self):
        """Request data from Google's Related Queries section and return a dictionary of dataframes
        If no top and/or rising related queries are found, the value for the key "top" and/or "rising" will be None
        """

        # make the request
        related_payload = dict()
        result_dict = dict()
        for request_json in self.related_queries_widget_list:
            # ensure we know which keyword we are looking at rather than relying on order
            try:
                kw = request_json['request']['restriction'][
                    'complexKeywordsRestriction']['keyword'][0]['value']
            except KeyError:
                kw = ''
            # convert to string as requests will mangle
            related_payload['req'] = json.dumps(request_json['request'])
            related_payload['token'] = request_json['token']
            related_payload['tz'] = self.tz

            # parse the returned json
            req_json = self._get_data(
                url=TrendReq.RELATED_QUERIES_URL,
                method=TrendReq.GET_METHOD,
                trim_chars=5,
                params=related_payload,
            )

            # top queries
            try:
                top_df = pd.DataFrame(
                    req_json['default']['rankedList'][0]['rankedKeyword'])
                top_df = top_df[['query', 'value']]
            except KeyError:
                # in case no top queries are found, the lines above will throw a KeyError
                top_df = None

            # rising queries
            try:
                rising_df = pd.DataFrame(
                    req_json['default']['rankedList'][1]['rankedKeyword'])
                rising_df = rising_df[['query', 'value']]
            except KeyError:
                # in case no rising queries are found, the lines above will throw a KeyError
                rising_df = None

            result_dict[kw] = {'top': top_df, 'rising': rising_df}
        return result_dict

    def trending_searches(self, pn='united_states'):
        """Request data from Google's Hot Searches section and return a dataframe"""

        # make the request
        # forms become obsolete due to the new TRENDING_SEARCHES_URL
        # forms = {'ajax': 1, 'pn': pn, 'htd': '', 'htv': 'l'}
        req_json = self._get_data(
            url=TrendReq.TRENDING_SEARCHES_URL,
            method=TrendReq.GET_METHOD
        )[pn]
        result_df = pd.DataFrame(req_json)
        return result_df

    def today_searches(self, pn='US'):
        """Request data from Google Daily Trends section and returns a dataframe"""
        forms = {'ns': 15, 'geo': pn, 'tz': '-180', 'hl': 'en-US'}
        req_json = self._get_data(
            url=TrendReq.TODAY_SEARCHES_URL,
            method=TrendReq.GET_METHOD,
            trim_chars=5,
            params=forms,
            **self.requests_args
        )['default']['trendingSearchesDays'][0]['trendingSearches']
        # parse the returned json
        result_df = pd.DataFrame(trend['title'] for trend in req_json)
        return result_df.iloc[:, -1]

    def realtime_trending_searches(self, pn='US', cat='all', count =300):
        """Request data from Google Realtime Search Trends section and returns a dataframe"""
        # Don't know what some of the params mean here, followed the nodejs library
        # https://github.com/pat310/google-trends-api/ 's implemenration


        #sort: api accepts only 0 as the value, optional parameter

        # ri: number of trending stories IDs returned,
        # max value of ri supported is 300, based on emperical evidence

        ri_value = 300
        if count < ri_value:
            ri_value = count

        # rs : don't know what is does but it's max value is never more than the ri_value based on emperical evidence
        # max value of ri supported is 200, based on emperical evidence
        rs_value = 200
        if count < rs_value:
            rs_value = count-1

        forms = {'ns': 15, 'geo': pn, 'tz': '300', 'hl': 'en-US', 'cat': cat, 'fi' : '0', 'fs' : '0', 'ri' : ri_value, 'rs' : rs_value, 'sort' : 0}
        req_json = self._get_data(
            url=TrendReq.REALTIME_TRENDING_SEARCHES_URL,
            method=TrendReq.GET_METHOD,
            trim_chars=5,
            params=forms
        )['storySummaries']['trendingStories']

        # parse the returned json
        wanted_keys = ["entityNames", "title"]

        final_json = [{ key: ts[key] for key in ts.keys() if key in wanted_keys} for ts in req_json ]

        result_df = pd.DataFrame(final_json)

        return result_df

    def top_charts(self, date, hl='en-US', tz=300, geo='GLOBAL'):
        """Request data from Google's Top Charts section and return a dataframe"""

        try:
            date = int(date)
        except:
            raise ValueError(
                'The date must be a year with format YYYY. See https://github.com/GeneralMills/pytrends/issues/355')

        # create the payload
        chart_payload = {'hl': hl, 'tz': tz, 'date': date, 'geo': geo,
                         'isMobile': False}

        # make the request and parse the returned json
        req_json = self._get_data(
            url=TrendReq.TOP_CHARTS_URL,
            method=TrendReq.GET_METHOD,
            trim_chars=5,
            params=chart_payload
        )
        try:
            df = pd.DataFrame(req_json['topCharts'][0]['listItems'])
        except IndexError:
            df = None
        return df

    def suggestions(self, keyword):
        """Request data from Google's Keyword Suggestion dropdown and return a dictionary"""

        # make the request
        kw_param = quote(keyword)
        parameters = {'hl': self.hl}

        req_json = self._get_data(
            url=TrendReq.SUGGESTIONS_URL + kw_param,
            params=parameters,
            method=TrendReq.GET_METHOD,
            trim_chars=5
        )['default']['topics']
        return req_json

    def categories(self):
        """Request available categories data from Google's API and return a dictionary"""

        params = {'hl': self.hl}

        req_json = self._get_data(
            url=TrendReq.CATEGORIES_URL,
            params=params,
            method=TrendReq.GET_METHOD,
            trim_chars=5
        )
        return req_json

    def get_historical_interest(self, *args, **kwargs):
        raise NotImplementedError(
            """This method has been removed for incorrectness. It will be removed completely in v5.
If you'd like similar functionality, please try implementing it yourself and consider submitting a pull request to add it to pytrends.

There is discussion at:
https://github.com/GeneralMills/pytrends/pull/542"""
        )

In [6]:
!pip show pytrends

Name: pytrends
Version: 4.9.2
Summary: Pseudo API for Google Trends
Home-page: 
Author: John Hogue, Burton DeWilde
Author-email: dreyco676@gmail.com
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: lxml, pandas, requests
Required-by: 


##CARGAR DATOS

In [7]:
url = 'https://raw.githubusercontent.com/juangui96/DS-_-Boot-Camp/main/Proyecto_Final/Actividad_1/super%20Store%20.csv'
ss = pd.read_csv(url, sep = ';')
ss['Order Date']=pd.to_datetime(ss['Order Date'],format='%d/%m/%Y')
ss['Mes']=ss['Order Date'].dt.month
ss['Year'] = ss['Order Date'].dt.year
ss.head(3)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Mes,Year
0,1,CA-2017-152156,2017-11-08,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,11,2017
1,2,CA-2017-152156,2017-11-08,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94,11,2017
2,3,CA-2017-138688,2017-06-12,16/06/2017,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.62,6,2017


##EDA
Como en entregas anteriores, se limpian:
- NA
- Se revisan vacios
- Se cambia el formato de las ventas (dado el separador que traer por defecto el DF)

In [8]:
ss.replace(0,np.NaN,inplace=True)

In [9]:
#validar NA
for i in range(len(ss['Sales'])):
  if pd.isna(ss['Sales'][i]) == True:
    ss['Sales'][i] = "0"
  if pd.isna(ss['Postal Code'][i]) == True:
    ss['Postal Code'][i]= "0"

ss.isna().sum()

Row ID           0
Order ID         0
Order Date       0
Ship Date        0
Ship Mode        0
Customer ID      0
Customer Name    0
Segment          0
Country          0
City             0
State            0
Postal Code      0
Region           0
Product ID       0
Category         0
Sub-Category     0
Product Name     0
Sales            0
Mes              0
Year             0
dtype: int64

In [10]:
#validar vacios
ss.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9799 entries, 0 to 9798
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Row ID         9799 non-null   int64         
 1   Order ID       9799 non-null   object        
 2   Order Date     9799 non-null   datetime64[ns]
 3   Ship Date      9799 non-null   object        
 4   Ship Mode      9799 non-null   object        
 5   Customer ID    9799 non-null   object        
 6   Customer Name  9799 non-null   object        
 7   Segment        9799 non-null   object        
 8   Country        9799 non-null   object        
 9   City           9799 non-null   object        
 10  State          9799 non-null   object        
 11  Postal Code    9799 non-null   object        
 12  Region         9799 non-null   object        
 13  Product ID     9799 non-null   object        
 14  Category       9799 non-null   object        
 15  Sub-Category   9799 n

In [11]:
#cambiando formato de sales a millones
ss['Sales2'] = ss['Sales'].str.replace('.','').astype("float")/1000000
ss.head(4)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Mes,Year,Sales2
0,1,CA-2017-152156,2017-11-08,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,11,2017,0.026196
1,2,CA-2017-152156,2017-11-08,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94,11,2017,0.073194
2,3,CA-2017-138688,2017-06-12,16/06/2017,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.62,6,2017,0.001462
3,4,US-2016-108966,2016-10-11,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,9.575.775,10,2016,9.575775


## ANALISIS DF VS APIS
Para aprovechar la informacion que se tiene en el DF, se propone revisar en google trends las tendencias y keywords de busqueda por zona geografica(estado) y por tiempo. Para realizar esto, se siguen los siguientes pasos:

1.   Definir Keywords
2.   Analizar la evolucion en el tiempo y zona geografica del interes de estas keywords
3.   Se realiza analisis geografico vs tiempo


#Definicion Key Words

Se crea una lista con las categorias y subcategorias del DF, posteriormente y para hacer el analisis mas real, se propone comporar cuales son las categorias y subcategorias que mas generan ingresos y luego revisar si son las que mas se buscan en google.

1. Categorias con mas ingresos: Furniture , Technology , Office Supplies
2. Subcategorias con mas ingresos: Bookcases, Chair, Phone, Tables,Binders, Storage y Copiers

In [12]:
#listas a buscar
kw_c = list(ss['Category'].unique())
kw_s = list(ss['Sub-Category'].unique())
kw_c
kw_s

['Bookcases',
 'Chairs',
 'Labels',
 'Tables',
 'Storage',
 'Furnishings',
 'Art',
 'Phones',
 'Binders',
 'Appliances',
 'Paper',
 'Accessories',
 'Envelopes',
 'Fasteners',
 'Supplies',
 'Machines',
 'Copiers']

In [13]:
#ventas de caregorias / subcategorias

v_c = pd.DataFrame(ss.groupby(['Category'])['Sales2'].sum()).reset_index()
v_c

,Category,Sales2
0,Furniture,542.881104
1,Office Supplies,250.545009
2,Technology,319.937278


In [14]:
v_s = pd.DataFrame(ss.groupby(['Sub-Category'])['Sales2'].sum()).reset_index().sort_values(by='Sales2',ascending = False)
v_s['pareto']=v_s['Sales2']/(v_s['Sales2'].sum())
v_s

,Sub-Category,Sales2,pareto
4,Bookcases,198.019806,0.177857
5,Chairs,181.085736,0.162647
13,Phones,160.430598,0.144095
16,Tables,136.495172,0.122597
3,Binders,102.579053,0.092134
14,Storage,64.148985,0.057617
6,Copiers,63.044102,0.056625
11,Machines,59.424089,0.053373
0,Accessories,37.038489,0.033267
9,Furnishings,27.280390,0.024503


##Busquedas relacionadas a las categorias de producto
Mediante la funcion pyptrends suggestions, se muestran sugerencias de palabras clave y de canales de promocion asociadas a las busquedas que hacen los clientes en google de las principales categorias de productos: Furniture, Technology y Office Supplies. Se resalta que:

*  Para Furniture existe una empresa que se llama Bobs Discount Furniture, la cual al parecer esta posicionada y es referente para los clientes. Asi mismo, se puede ver que los elementos de las salas (living room) son los que mas se buscan en Google, seguido de baños, por lo cual es preciso revisar si alguna linea de productos esta enfocada en estos segmentos.
*  Para office supplies, hay una empresa que es referente: Fournitures De Bureau Denis Inc. Esta posicionada y al igual que el caso anterior es reconocida Se podria realizar un analisis sobre su movimiento comerciales para adquirir buenas practicas.
*  Para technology se evidencia que hay un canal en Youtube que es altamente buscado, por lo cual podria revisarse porque los clientes acuden alli y si hay alguna manera de hacer algun tipo de promocion para posicionar los productos.


In [15]:
pytrends= TrendReq(hl='en',retries=3, backoff_factor=20)

In [16]:
kw_c

['Furniture', 'Office Supplies', 'Technology']

## Busqueda Subcategorias
Al realizar el mismo analisis anterior pero ahora llevado a la dimension de subcategorias, se encuentra que son bastantes por lo cual se toman las principales  (las que generan un mayor ingreso) y se encuentra los siguiente:

*  Para bookcases hay un busquedas recurrentes en cuanto a este producto pero enfocado a niños, asi mismo se puede ver los usuarios para encontrar informacion sobre el producto utilizan el termino: Bookcases & Standing Shelves, por lo cual es importante garantizar que la empresa aparezca cuando los usuarios utilicen medios digitales con esas keywords.
*  La lineas de productos de mesas, deben estar orientadas a las principales busquedas que se mencionan, entre ellas: poker, campamento, picnic entre otros. se debe revisar si hay modelos que entren dentro de estas categorias para poder adapatarlos y que aparezcan en las busquedas.
*  Binders es una categoria que es bastante amplia, aun mas que la descrita anteriormente - mesas -. Se debe identificar que tipo de carpetas son las que se tienen y revisar si estan orientadas a lo que los clientes buscan.

In [17]:
data_frame_f=pd.DataFrame()

for i in kw_s:
  data_frame = pytrends.suggestions(i)
  data_frame = pd.DataFrame(data_frame)
  data_frame['kw'] = i
  data_frame_f.iloc[0:3]
  data_frame_f = pd.concat([data_frame_f,data_frame],axis=0)

data_frame_f

,mid,title,type,kw
0,/m/03__z0,Bookcase,Topic,Bookcases
1,/g/11h1hvqv6z,Kids' Bookcase,Topic,Bookcases
2,/g/11s659t822,Days at the Morisaki Bookshop: A Novel,Book by Satoshi Yagisawa,Bookcases
3,/g/11f2_70z5f,Bookcases & Standing Shelves,Topic,Bookcases
4,/g/11hz1nw25r,Swedish Death Cleaning Workbook: The 30 Days C...,Book by Kelsie Zaria,Bookcases
...,...,...,...,...
4,/m/0dkw5,Machine,Topic,Machines
0,/m/01gx4y,Photocopier,Topic,Copiers
1,/g/11gdpg02vz,"Printers, Copiers & Fax Machines",Topic,Copiers
2,/g/11b5l_zhmv,Winners Dream: A Journey from Corner Store to ...,Book by Bill McDermott,Copiers


In [18]:
data_frame_f1=pd.DataFrame()

for i in kw_c:
  data_frame = pytrends.suggestions(i)
  data_frame = pd.DataFrame(data_frame)
  data_frame['kw'] = i
  data_frame_f1 = pd.concat([data_frame_f1,data_frame],axis=0)

data_frame_f1

,mid,title,type,kw
0,/g/11gfn_17d9,Bob's Discount Furniture,Topic,Furniture
1,/m/0c_jw,Furniture,Topic,Furniture
2,/g/11h1hvzzd8,Living Room Furniture,Topic,Furniture
3,/g/12214lv0m,Office Furniture,Topic,Furniture
4,/g/11fl7dz2f3,Bathroom Furniture,Topic,Furniture
0,/g/11g6qmgd1d,Office Supplies,Topic,Office Supplies
1,/g/11h79tzll4,Office Supplies,Topic,Office Supplies
2,/m/02rdsp,Office supplies,Topic,Office Supplies
3,/g/11fy2bpvy1,Fournitures De Bureau Denis Inc.,Company,Office Supplies
0,/m/07c1v,Technology,Topic,Technology


In [19]:
pkw_s = ["Bookcases", "Chair", "Phone", "Tables","Binders", "Storage" ,"Copiers"]
data_frame_f2 = data_frame_f[data_frame_f['kw'].isin(pkw_s)]
data_frame_f2

,mid,title,type,kw
0,/m/03__z0,Bookcase,Topic,Bookcases
1,/g/11h1hvqv6z,Kids' Bookcase,Topic,Bookcases
2,/g/11s659t822,Days at the Morisaki Bookshop: A Novel,Book by Satoshi Yagisawa,Bookcases
3,/g/11f2_70z5f,Bookcases & Standing Shelves,Topic,Bookcases
4,/g/11hz1nw25r,Swedish Death Cleaning Workbook: The 30 Days C...,Book by Kelsie Zaria,Bookcases
0,/m/07cwnp,Picnic table,Topic,Tables
1,/m/0dmsh,Drawing board,Topic,Tables
2,/m/027b95w,Massage table,Topic,Tables
3,/g/11f2_6rbzm,Accent & End Tables,Topic,Tables
4,/g/11jcmcqn1d,Camp Table,Topic,Tables


# Interes en el tiempo


## Categorias en el tiempo
Al revisar la evolucion de las categoerias en el tiempo se evidencia que lo mas búsqueda es furniture y por un amplia diferencia, se ve que la tendencia es cíclica y que todos los meses de noviembre las búsquedas bajan luego tener un pico bastante marcado los días 19 y 18 de ese mes. Asi mismo, se evidencia que los meses de mayo y junio tienden a tener un interés bajo mientras que los primeros meses del año sube. Por otro lado, los picos con menor cantidad de búsquedas corresponden a la semana del 24 de diciembre y no solo para la caegoria de furniture sino también para technology . de esta ultima se puede decir que es un poco mas estable que la de furniture, sine mbargo también presenta un comportamiento ciclico. Por ultimo, office supplies es la que menos se busca, por lo cual se puede pensar que esta no tiene gran presencial en el canal digital.

In [20]:
# pip show requeridos para correr las funciones
!pip show pytrends
!pip show requests
!pip show json


Name: pytrends
Version: 4.9.2
Summary: Pseudo API for Google Trends
Home-page: 
Author: John Hogue, Burton DeWilde
Author-email: dreyco676@gmail.com
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: lxml, pandas, requests
Required-by: 
Name: requests
Version: 2.31.0
Summary: Python HTTP for Humans.
Home-page: https://requests.readthedocs.io
Author: Kenneth Reitz
Author-email: me@kennethreitz.org
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: certifi, charset-normalizer, idna, urllib3
Required-by: bigframes, CacheControl, chart-studio, community, earthengine-api, fastai, folium, gcsfs, gdown, geocoder, google-api-core, google-cloud-bigquery, google-cloud-storage, google-colab, huggingface-hub, kaggle, kagglehub, moviepy, music21, pandas-datareader, panel, pooch, pymystem3, pytrends, requests-oauthlib, spacy, Sphinx, tensorboard, tensorflow-datasets, torchdata, torchtext, transformers, tweepy, weasel, yfinance


In [21]:
#definicion de funcion , idioma, kewywords, pais y tiempo de busqueda
#categorias
pytrends= TrendReq(hl='en',retries=3, backoff_factor=20)
keywords = kw_c
pytrends.build_payload(keywords, cat=0, timeframe='2015-01-01 2018-12-31', geo='US', gprop='')
#funcion busqueda interes en el tiempo
int_cat_1=(pytrends.interest_over_time())
int_cat_1=int_cat_1.drop(columns='isPartial')

In [ ]:
int_cat_1.info()
px.line(int_cat_1)

## Subcategorias en el tiempo
Al igual que el comportamiento por categorías, al evaluar las subcategorías de los productos se evidencia que la tendencia la marcan los teléfonos celulares, lo primero que es importante mencionar es que hay una tendencia a aumentar en el tiempo, ademas al revisar la grafica se muestra que, si bien hay mínimos de interés en los meses de noviembre y abril, el aumento vuelve a presentarse durante el segundo semestre de cada año. Asi mismo se ve que la categoría sillas, meseas y binders presentar un interés menor al 20. Por lo cual, es oportuno preguntarse si existen algunos productos asociados a furniture que no se estén mostrando y que puedan explotarse en un canal digital, dado que las personas si buscan el termino “furniture” como se pudo apreciar anteriormente

In [23]:
#definicion de funcion , idioma, kewywords, pais y tiempo de busqueda
#sub categorias
pytrends= TrendReq(hl='en',retries=3, backoff_factor=20)
keywords = ['Bookcases', 'Chair', 'Phone', 'Tables', 'Binders']
pytrends.build_payload(keywords, cat=0, timeframe='2015-01-01 2018-12-31', geo='US', gprop='')
#funcion busqueda interes en el tiempo
int_cat=pytrends.interest_over_time()
int_cat

,Bookcases,Chair,Phone,Tables,Binders,isPartial
date,,,,,,
2014-12-28,0,8,84,4,0,False
2015-01-04,0,7,84,4,0,False
2015-01-11,0,8,83,4,0,False
2015-01-18,0,8,82,5,0,False
2015-01-25,0,8,82,5,0,False
...,...,...,...,...,...,...
2018-12-02,0,11,93,4,0,False
2018-12-09,0,11,89,4,0,False
2018-12-16,0,11,94,4,0,False


In [24]:
int_cat=int_cat.drop(columns='isPartial')
px.line(int_cat)

#Interes por region
El analisis de interes de los principales productos se centra en identificar las regiones que presentan mayores ventas y un interes importante en cierta categoria de producto, en este caso Chiar y Phone. Como se puede apreciar en la grafica, los puntos mas grandes son los que tienen un total de ventas mayor, de esta manera se identifica que:
Texas, California y NY, son las mejores plazas, seguidas de Ohio, Florida entre otros. En cuanto al interes, a nivel general, Phone marca tendencia teniendo una media de 86, en las regiones anteriormente mencionadas se destaca Texas con un interes del 87. En cuanto a la categoria Chair, NY es el estado en donde mas se busca este termino seguido de California.
Las demas categorias, Book y Blinders, no tienen un interes tan alto.

In [25]:
!pip show requests
!pip show json

Name: requests
Version: 2.31.0
Summary: Python HTTP for Humans.
Home-page: https://requests.readthedocs.io
Author: Kenneth Reitz
Author-email: me@kennethreitz.org
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: certifi, charset-normalizer, idna, urllib3
Required-by: bigframes, CacheControl, chart-studio, community, earthengine-api, fastai, folium, gcsfs, gdown, geocoder, google-api-core, google-cloud-bigquery, google-cloud-storage, google-colab, huggingface-hub, kaggle, kagglehub, moviepy, music21, pandas-datareader, panel, pooch, pymystem3, pytrends, requests-oauthlib, spacy, Sphinx, tensorboard, tensorflow-datasets, torchdata, torchtext, transformers, tweepy, weasel, yfinance


In [26]:
pytrends= TrendReq(hl='en',retries=3, backoff_factor=20)
keywords = ['Bookcases', 'Chair', 'Phone', 'Tables', 'Binders']
pytrends.build_payload(keywords, cat=0, timeframe='2015-01-01 2018-12-31', geo='US', gprop='')
ior=pytrends.interest_by_region()
ior

,Bookcases,Chair,Phone,Tables,Binders
geoName,,,,,
Alabama,0,8,89,3,0
Alaska,0,9,87,4,0
Arizona,0,9,87,4,0
Arkansas,0,8,89,3,0
California,0,10,86,4,0
Colorado,0,10,85,5,0
Connecticut,0,11,84,5,0
Delaware,0,10,86,4,0
District of Columbia,0,11,85,4,0


In [27]:
ior = ior.sort_values(by='Phone', ascending = False )
ior = pd.DataFrame(ior).reset_index()
ior.columns.values

array(['geoName', 'Bookcases', 'Chair', 'Phone', 'Tables', 'Binders'],
      dtype=object)

In [60]:
ingresos = ss.groupby('State')['Sales2'].sum().sort_values(ascending= False).reset_index()
int_sales = ior.merge(ingresos, right_on='State',left_on='geoName')
int_sales = int_sales.drop(columns=['geoName'])
int_sales
px.scatter(int_sales,x='State',y='Chair',size='Phone',color='Sales2')

In [61]:
px.scatter(int_sales,x='State',y='Phone',size='Chair',color='Sales2')

#Busqueda de Nombre de Productos en Google
Para revisar si alguno de los productos ha sido tendencia en busquedas en google, se realiza primero un analisis a partir de un scatter plot, en donde el tamaño de cada punto se explica por la cantidad total de ventas de cada articulo, como se puede apreciar casi todos tienen el mismo tamaño, sin embargo, en el eje y se grafica las ventas totales en dinero, y se puede apreciar que la gran mayoria estan por debajo de 5 millones. Para mejorar el entendimiento se realiza nuevamente el scatter plot pero unicamente con los productos que vendieron mas de 5 millones, en este caso, se ve que la mayorita tiene un tamaño similar (catidad de ventas) pero pocos pasan los 15 millones de unidades vendidas.
Paralelo a lo anterior, se realiza un grafico de barras combinado con un mapa de calor, para conocer las referencias que mas ventas tienen y mas unidades venden. De esta manera se escogen los que venden mayor a 20 y tienen mas de 6 ventas ademas de los productos que si bien no tiene unas ventas altas si tienen una gran cantidad de unidades venidas (mas de 12, correspondientes a los 3 ultimos de la siguiente lista).
De esta manera se identifican los siguientes productos:

'Riverside Palais Royal Lawyers Bookcase, Royale Cherry Finish',
 'Atlantic Metals Mobile 5-Shelf Bookcases, Custom Colors',
 'Canon imageCLASS 2200 Advanced Copier',
 "O'Sullivan Plantations 2-Door Library in Landvery Oak",
 'GBC DocuBind TL300 Electric Binding System',
 'Chromcraft Round Conference Tables',
 'Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back'


Al revisar las tendencias de estos articulos se evidencia que no es ciclico su comportamiento ni tampoco tiene estacionalidad, ademas a nivel general, se ve que existen periodos de tiempo muy puntuales en donde estos fueron tendencia, siendo el Primer (Riverside Palais Royal Lawyers Bookcase, Royale Cherry Finish)
el que mas picos de interes tiene, junto con Atlantic Metals Mobile 5-Shelf Bookcases, Custom Colors ambos en los mismos periodos: Mayo de 2015, Junio 2015 y Noviembre de 2015, se puede pensar que fueron efectos de publicidad y de colocacion de la marca dado que esto soslo se presenta en el primer año.
Por otro lado el producto Canon imageCLASS 2200 Advanced Copier unicamente fue tendencia en junio de 2017 y por ultimo el O'Sullivan Plantations 2-Door Library in Landvery Oak presento mayor cantidad de buquedas en el primer semestre de 2015 sin embargo nunca mas estuvo en el top de busquedas, puede ser que este producto fue muy popular al inicio y luego fue perdiendo su posicionamiento.
De los ultimos 3 productos, solo se resalta que el que realmente fue tendencia en 2016 corresponde a Chromcraft Round Conference Tables, sin embargo tuvo solo un par de picos de resto su interes, al igual que los demas articulos, es de 0.

In [62]:
v_np = pd.DataFrame(ss.groupby(['Product Name','Product ID'])['Sales2'].agg(['count','sum'])).reset_index().sort_values(by='sum',ascending = False)
v_np.dtypes

Product Name     object
Product ID       object
count             int64
sum             float64
dtype: object

In [63]:
px.scatter(v_np,x='Product ID',y='sum',size='count')

In [64]:
v_np2 = v_np[v_np['sum']>5]
px.scatter(v_np2,x='Product ID',y='sum',size='count')

In [65]:
px.bar(v_np2,x='Product ID',y='sum',color='count',barmode = 'group')

In [131]:
v_np3 = v_np2[(v_np2['sum']>20)]
v_np3

,Product Name,Product ID,count,sum
1364,"Riverside Palais Royal Lawyers Bookcase, Royal...",FUR-BO-10004834,5,30.041418
170,"Atlantic Metals Mobile 5-Shelf Bookcases, Cust...",FUR-BO-10001811,8,23.912861
406,Canon imageCLASS 2200 Advanced Copier,TEC-CO-10004722,5,23.799932
1202,O'Sullivan Plantations 2-Door Library in Landv...,FUR-BO-10000780,5,23.585003


In [138]:
pytrends= TrendReq(hl='en',retries=3, backoff_factor=20)
kw_nombre = v_np3['Product Name']
pytrends.build_payload(kw_nombre, cat=0, timeframe='2015-01-01 2018-12-31', geo='US', gprop='')
#funcion busqueda interes en el tiempo
int_pro=pytrends.interest_over_time()
int_pro = int_pro.drop(columns='isPartial')
int_pro
px.line(int_pro)

In [116]:
v_np4 =v_np2[v_np2['Product ID'].str.endswith('1359')|
                                 v_np2['Product ID'].str.endswith('1095')|
                                 v_np2['Product ID'].str.endswith('454')]
v_np4

,Product Name,Product ID,count,sum
695,GBC DocuBind TL300 Electric Binding System,OFF-BI-10001359,11,10.943278
430,Chromcraft Round Conference Tables,FUR-TA-10001095,13,5.542422
848,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",FUR-CH-10000454,12,5.257769


In [139]:
pytrends= TrendReq(hl='en',retries=3, backoff_factor=20)
kw_nombre1 = v_np4['Product Name']
pytrends.build_payload(kw_nombre1, cat=0, timeframe='2015-01-01 2018-12-31', geo='US', gprop='')
#funcion busqueda interes en el tiempo
int_pro1=pytrends.interest_over_time()
int_pro1 = int_pro1.drop(columns='isPartial')
px.line(int_pro1)

En la siguiente seccion se comparan los meses de interes donde los principales productos fueron tendencia, sin embargo, como se evidencia para esos meses no se presento venta de estos productos por lo cual, se puede inferir que la tendencia no influye en el precio para este grupo de productos

In [169]:
prd_vs_busq = list(kw_nombre)+list(kw_nombre1)

In [171]:
int_pro.columns.values

array(['Riverside Palais Royal Lawyers Bookcase, Royale Cherry Finish',
       'Atlantic Metals Mobile 5-Shelf Bookcases, Custom Colors',
       'Canon imageCLASS 2200 Advanced Copier',
       "O'Sullivan Plantations 2-Door Library in Landvery Oak"],
      dtype=object)

In [260]:
#filtrar los productos que no tenian un interes >0 para los productos previamente mencionados
int_pro_f = pd.concat([int_pro, int_pro1], axis=0)
int_pro_f.replace(np.nan, 0, inplace=True)
int_pro_f['suma'] = int_pro_f['Riverside Palais Royal Lawyers Bookcase, Royale Cherry Finish'] + \
                    int_pro_f['Atlantic Metals Mobile 5-Shelf Bookcases, Custom Colors'] + \
                    int_pro_f['Canon imageCLASS 2200 Advanced Copier'] + \
                    int_pro_f["O'Sullivan Plantations 2-Door Library in Landvery Oak"] + \
                    int_pro_f['GBC DocuBind TL300 Electric Binding System'] + \
                    int_pro_f['Chromcraft Round Conference Tables'] + \
                    int_pro_f['Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back']
int_pro_f = int_pro_f[int_pro_f['suma']>0]
int_pro_f = pd.DataFrame(int_pro_f).reset_index()

In [261]:
#Extraer la feha mes en que fue tendencia de cada producto
int_pro_f['mes_año'] = int_pro_f['date'].dt.strftime('%m/%Y')
int_pro_f = int_pro_f.drop(columns=['date','suma'])
int_pro_f

,"Riverside Palais Royal Lawyers Bookcase, Royale Cherry Finish","Atlantic Metals Mobile 5-Shelf Bookcases, Custom Colors",Canon imageCLASS 2200 Advanced Copier,O'Sullivan Plantations 2-Door Library in Landvery Oak,GBC DocuBind TL300 Electric Binding System,Chromcraft Round Conference Tables,"Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back",mes_año
0,0.0,0.0,0.0,73.0,0.0,0.0,0.0,01/2015
1,0.0,0.0,0.0,90.0,0.0,0.0,0.0,03/2015
2,89.0,0.0,0.0,0.0,0.0,0.0,0.0,05/2015
3,0.0,80.0,0.0,0.0,0.0,0.0,0.0,05/2015
4,0.0,85.0,0.0,0.0,0.0,0.0,0.0,10/2015
5,80.0,0.0,0.0,0.0,0.0,0.0,0.0,11/2015
6,0.0,100.0,0.0,0.0,0.0,0.0,0.0,11/2015
7,0.0,0.0,65.0,0.0,0.0,0.0,0.0,01/2017
8,0.0,55.0,0.0,0.0,0.0,0.0,0.0,11/2018
9,0.0,0.0,0.0,0.0,0.0,100.0,0.0,10/2015


In [265]:
#convertir filas en columnas - melt
int_pro_f2 = int_pro_f.melt(
    id_vars = ['mes_año'],
    var_name = 'Producto',
    value_name='Interes')
int_pro_f2 = int_pro_f2[int_pro_f2['Interes'] > 0]
int_pro_f2

,mes_año,Producto,Interes
2,05/2015,"Riverside Palais Royal Lawyers Bookcase, Royal...",89.0
5,11/2015,"Riverside Palais Royal Lawyers Bookcase, Royal...",80.0
14,05/2015,"Atlantic Metals Mobile 5-Shelf Bookcases, Cust...",80.0
15,10/2015,"Atlantic Metals Mobile 5-Shelf Bookcases, Cust...",85.0
17,11/2015,"Atlantic Metals Mobile 5-Shelf Bookcases, Cust...",100.0
19,11/2018,"Atlantic Metals Mobile 5-Shelf Bookcases, Cust...",55.0
29,01/2017,Canon imageCLASS 2200 Advanced Copier,65.0
33,01/2015,O'Sullivan Plantations 2-Door Library in Landv...,73.0
34,03/2015,O'Sullivan Plantations 2-Door Library in Landv...,90.0
64,10/2015,Chromcraft Round Conference Tables,100.0


In [269]:
#Creacion de llaves _ df interes
int_pro_f2['llave']=int_pro_f2['Producto']+int_pro_f2['mes_año'].astype("str")
int_pro_f2

,mes_año,Producto,Interes,llave
2,05/2015,"Riverside Palais Royal Lawyers Bookcase, Royal...",89.0,"Riverside Palais Royal Lawyers Bookcase, Royal..."
5,11/2015,"Riverside Palais Royal Lawyers Bookcase, Royal...",80.0,"Riverside Palais Royal Lawyers Bookcase, Royal..."
14,05/2015,"Atlantic Metals Mobile 5-Shelf Bookcases, Cust...",80.0,"Atlantic Metals Mobile 5-Shelf Bookcases, Cust..."
15,10/2015,"Atlantic Metals Mobile 5-Shelf Bookcases, Cust...",85.0,"Atlantic Metals Mobile 5-Shelf Bookcases, Cust..."
17,11/2015,"Atlantic Metals Mobile 5-Shelf Bookcases, Cust...",100.0,"Atlantic Metals Mobile 5-Shelf Bookcases, Cust..."
19,11/2018,"Atlantic Metals Mobile 5-Shelf Bookcases, Cust...",55.0,"Atlantic Metals Mobile 5-Shelf Bookcases, Cust..."
29,01/2017,Canon imageCLASS 2200 Advanced Copier,65.0,Canon imageCLASS 2200 Advanced Copier01/2017
33,01/2015,O'Sullivan Plantations 2-Door Library in Landv...,73.0,O'Sullivan Plantations 2-Door Library in Landv...
34,03/2015,O'Sullivan Plantations 2-Door Library in Landv...,90.0,O'Sullivan Plantations 2-Door Library in Landv...
64,10/2015,Chromcraft Round Conference Tables,100.0,Chromcraft Round Conference Tables10/2015


In [271]:
#Creacion de llaves _ ss_interes
ss["mes_año"] = ss['Order Date'].dt.strftime("%m/%Y")
ss_int_pro = pd.DataFrame(ss.groupby(['Product Name','mes_año'])['Sales2'].sum()).reset_index()
ss_int_pro = ss_int_pro[ss_int_pro['Product Name'].isin(prd_vs_busq)]
ss_int_pro['llave']=ss_int_pro['Product Name']+ss_int_pro['mes_año'].astype("str")
ss_int_pro

,Product Name,mes_año,Sales2,llave
841,"Atlantic Metals Mobile 5-Shelf Bookcases, Cust...",02/2018,0.240784,"Atlantic Metals Mobile 5-Shelf Bookcases, Cust..."
842,"Atlantic Metals Mobile 5-Shelf Bookcases, Cust...",03/2016,12.279984,"Atlantic Metals Mobile 5-Shelf Bookcases, Cust..."
843,"Atlantic Metals Mobile 5-Shelf Bookcases, Cust...",03/2018,1.023332,"Atlantic Metals Mobile 5-Shelf Bookcases, Cust..."
844,"Atlantic Metals Mobile 5-Shelf Bookcases, Cust...",05/2018,2.046664,"Atlantic Metals Mobile 5-Shelf Bookcases, Cust..."
845,"Atlantic Metals Mobile 5-Shelf Bookcases, Cust...",07/2017,1.279165,"Atlantic Metals Mobile 5-Shelf Bookcases, Cust..."
846,"Atlantic Metals Mobile 5-Shelf Bookcases, Cust...",09/2017,0.722352,"Atlantic Metals Mobile 5-Shelf Bookcases, Cust..."
847,"Atlantic Metals Mobile 5-Shelf Bookcases, Cust...",09/2018,0.180588,"Atlantic Metals Mobile 5-Shelf Bookcases, Cust..."
848,"Atlantic Metals Mobile 5-Shelf Bookcases, Cust...",11/2016,6.139992,"Atlantic Metals Mobile 5-Shelf Bookcases, Cust..."
1964,Canon imageCLASS 2200 Advanced Copier,03/2018,1.399996,Canon imageCLASS 2200 Advanced Copier03/2018
1965,Canon imageCLASS 2200 Advanced Copier,05/2017,8.399976,Canon imageCLASS 2200 Advanced Copier05/2017


In [286]:
#interesprd_vs_ingreso
int_prd_ing = pd.merge(ss_int_pro,int_pro_f2, how="right", left_on="llave",right_on='llave')
int_prd_ing

,Product Name,mes_año_x,Sales2,llave,mes_año_y,Producto,Interes
0,NaN,NaN,NaN,"Riverside Palais Royal Lawyers Bookcase, Royal...",05/2015,"Riverside Palais Royal Lawyers Bookcase, Royal...",89.0
1,NaN,NaN,NaN,"Riverside Palais Royal Lawyers Bookcase, Royal...",11/2015,"Riverside Palais Royal Lawyers Bookcase, Royal...",80.0
2,NaN,NaN,NaN,"Atlantic Metals Mobile 5-Shelf Bookcases, Cust...",05/2015,"Atlantic Metals Mobile 5-Shelf Bookcases, Cust...",80.0
3,NaN,NaN,NaN,"Atlantic Metals Mobile 5-Shelf Bookcases, Cust...",10/2015,"Atlantic Metals Mobile 5-Shelf Bookcases, Cust...",85.0
4,NaN,NaN,NaN,"Atlantic Metals Mobile 5-Shelf Bookcases, Cust...",11/2015,"Atlantic Metals Mobile 5-Shelf Bookcases, Cust...",100.0
5,NaN,NaN,NaN,"Atlantic Metals Mobile 5-Shelf Bookcases, Cust...",11/2018,"Atlantic Metals Mobile 5-Shelf Bookcases, Cust...",55.0
6,NaN,NaN,NaN,Canon imageCLASS 2200 Advanced Copier01/2017,01/2017,Canon imageCLASS 2200 Advanced Copier,65.0
7,NaN,NaN,NaN,O'Sullivan Plantations 2-Door Library in Landv...,01/2015,O'Sullivan Plantations 2-Door Library in Landv...,73.0
8,NaN,NaN,NaN,O'Sullivan Plantations 2-Door Library in Landv...,03/2015,O'Sullivan Plantations 2-Door Library in Landv...,90.0
9,NaN,NaN,NaN,Chromcraft Round Conference Tables10/2015,10/2015,Chromcraft Round Conference Tables,100.0
